# Demo for AIFlow class


In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
API_KEY=os.getenv("API_KEY")
from aiflow import AIFlow


# Just an empty flow

In [ ]:
flow1 = AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=0)
(
    flow1
    .show_model_config()
    .show_self_data()
    .pretty_print_messages()
    .pretty_print_messages_to_file(file_name="nothing.txt")
)

# Chat demo

In [ ]:
chatbot=AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=1)
chatbot.show_model_config().show_self_data()

In [ ]:
(    chatbot
        .show_model_config()
        .set_system_prompt("You are a helpful cooking assistant always eager to give a recipe for whatever the user says.")
        .add_user_chat("How is the weather here?")
        .show_self_data()
)

In [ ]:
chatbot.add_user_chat("Ok. Give me a recipe inspired by weather")

In [ ]:
chatbot.latest_to_context("dish")


In [ ]:
chatbot.show_self_data()

## Message manipulations

In [ ]:
chatbot.reduce_messages_to_text(lambda x: (print(x), "Replacing latest with all sorts of nothing")[1]).show_self_data()

In [ ]:
chatbot.filter_messages(lambda items: [{**item, "content": "You are an artist very good in poetry"} if item.get("role") == "system" else item for item in items])

In [ ]:
chatbot.show_self_data()

## Just completions - no chat

In [ ]:
chatbot.completion("Give me 10 random names")

In [ ]:
chatbot.show_self_data()

In [ ]:
chatbot.completion("Give the following names in reverse order: [latest].")

## Some context manipulations

In [ ]:
chatbot.show_self_data()

In [ ]:
chatbot.set_context(content="Nothing", label="stuff").show_self_data().delete_context(label="stuff").show_self_data()

## Data loading

In [ ]:
chatbot.read_textfile_to_context(filename="output.txt").show_self_data()

# Combining AIFlows

In [5]:
otherbot=AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=1)
otherbot.completion("Let's say you are a random person hungry for food. What would you like to eat?")

Let's say you are a random person hungry for food. What would you like to eat?

I would probably go for a classic cheeseburger with fries or maybe some pizza. Those are always reliable options for satisfying hunger.



In [6]:
otherbot.return_latest_to_text()

'I would probably go for a classic cheeseburger with fries or maybe some pizza. Those are always reliable options for satisfying hunger.'

In [11]:
sillybot=AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=1)
sillybot.set_context(content=otherbot.return_latest_to_text(), label="food_choice").show_self_data()
sillybot.completion(prompt="Give a silly comment on the following choice of food: [food_choice]").show_self_data()

Chat Messages:
[]

Context Map:
{
    "food_choice": "I would probably go for a classic cheeseburger with fries or maybe some pizza. Those are always reliable options for satisfying hunger."
}

Images Map:
{}

Audio Map:
{}
Give a silly comment on the following choice of food: [food_choice]

Why not both? Cheeseburger pizza! Double the deliciousness, double the fun!

Chat Messages:
[]

Context Map:
{
    "food_choice": "I would probably go for a classic cheeseburger with fries or maybe some pizza. Those are always reliable options for satisfying hunger.",
    "latest": "Why not both? Cheeseburger pizza! Double the deliciousness, double the fun!"
}

Images Map:
{}

Audio Map:
{}


# Using vectordatabase Chroma

In [15]:
# https://docs.trychroma.com/guides
import chromadb
chroma_client = chromadb.Client() # chromadb.PersistentClient(path="/path/to/save/to")
collection = chroma_client.create_collection(name="my_collection")

In [16]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)

In [17]:
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=1 # how many results to return
)
print(results)

{'ids': [['id1', 'id2']], 'distances': [[1.0404009819030762, 1.2430799007415771]], 'metadatas': [[None, None]], 'embeddings': None, 'documents': [['This is a document about pineapple', 'This is a document about oranges']], 'uris': None, 'data': None}


In [18]:
collection.peek() # returns a list of the first 10 items in the collection
collection.count() # returns the number of items in the collection

2

In [19]:
print(chroma_query_result_to_text(results))

This is a document about pineapple
This is a document about oranges


In [22]:
sillybot.set_context(content=chroma_query_result_to_text(results), label="found_doc")
sillybot.completion("Say something really silly about this random text: [found_doc]").show_self_data()

Say something really silly about this random text: [found_doc]

But don't get it twisted, we're not comparing apples to oranges, we're comparing pineapples to oranges!

Chat Messages:
[]

Context Map:
{
    "food_choice": "I would probably go for a classic cheeseburger with fries or maybe some pizza. Those are always reliable options for satisfying hunger.",
    "latest": "But don't get it twisted, we're not comparing apples to oranges, we're comparing pineapples to oranges!",
    "found_doc": "This is a document about pineapple\nThis is a document about oranges"
}

Images Map:
{}

Audio Map:
{}
